<a href="https://colab.research.google.com/github/guilhermelaviola/IntegrativePracticeInDataScience/blob/main/Class10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Learning Applications**
Deep Learning, a subfield of machine learning, excels in processing large datasets and solving complex challenges. Its prominent application is in generative models, like Generative Adversarial Networks (GANs), which create new data from existing datasets. Unlike generative models, sequential models predict data sequences and are used in demand forecasting and speech recognition, with Markov Chains being a key example. Deep Learning also transforms computer vision through convolutional neural networks (CNNs), aiding in object detection and segmentation for autonomous vehicles and medical diagnostics. Additionally, Natural Language Processing (NLP) enables machines to understand human language, facilitating automatic translation and chatbots. Overall, Deep Learning drives innovation across various fields, shaping the future of artificial intelligence.

In [1]:
# Importing al the necessary libraries and resources:
from tensorflow.keras import datasets, layers, models, Model
import random
from transformers import pipeline

## **CNN example with Keras**

In [2]:
# Loading MNIST digits dataset:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train[..., None] / 255.0
x_test = x_test[..., None] / 255.0

# Building a simple CNN:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1)
print('Test accuracy:', model.evaluate(x_test, y_test)[1])

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 41ms/step - accuracy: 0.9165 - loss: 0.2811
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9787 - loss: 0.0690
Test accuracy: 0.982200026512146


## **Markov Chain example**

In [4]:
# Transition probabilities:
transitions = {
    'sunny': {'sunny': 0.8, 'rainy': 0.2},
    'rainy': {'sunny': 0.4, 'rainy': 0.6},
}

def next_state(current):
    states = list(transitions[current].keys())
    probs = list(transitions[current].values())
    return random.choices(states, probs)[0]

# Generating a sequence:
state = 'sunny'
sequence = [state]

for _ in range(10):
    state = next_state(state)
    sequence.append(state)

print(sequence)

['sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'rainy', 'sunny', 'sunny', 'rainy', 'rainy', 'sunny']


## **Tiny GAN-like Architecture**

In [5]:
# Generator:
def build_generator():
    inputs = layers.Input(shape=(100,))
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(28 * 28, activation='sigmoid')(x)
    outputs = layers.Reshape((28, 28, 1))(x)
    return Model(inputs, outputs)

# Discriminator:
def build_discriminator():
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Flatten()(inputs)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs)

G = build_generator()
D = build_discriminator()

print('Generator:', G.summary())
print('Discriminator:', D.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 784)            │       101,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 114,064 (445.56 KB)

 Trainable params: 114,064 (445.56 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,609 (393.00 KB)

 Trainable params: 100,609 (393.00 KB)

 Non-trainable params: 0 (0.00 B)

Discriminator: None


## **NLP example**

In [6]:
classifier = pipeline('sentiment-analysis')
result = classifier('Deep learning is transforming AI and technology.')
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9996684789657593}]
